<a href="https://colab.research.google.com/github/Carinaaa/ML-Learning-Path/blob/intro-LLM/LLMs_summary_openAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/api-keys
https://stackoverflow.com/questions/66631333/how-do-i-set-environment-variables-in-google-colab

https://github.com/ed-donner/llm_engineering


In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
from google.colab import userdata
import os
# load_dotenv() # load the .env file if you have it, i don't and I use secrets from GC
openai_key = userdata.get('OPENAI_API_KEY') # get the secret by the name
os.environ['OPENAI_API_KEY'] = openai_key # set it as an env var
openai = OpenAI() # create model using the env var

In [ ]:
message = "Hello, GPT! This is my first ever message to you! Hi!"

response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])

print(response.choices[0].message.content)

In [ ]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
History_of_Christianity_wiki = Website("https://en.wikipedia.org/wiki/History_of_Christianity")
print(History_of_Christianity_wiki.title)
print(History_of_Christianity_wiki.text)

## Types of prompts

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [ ]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(History_of_Christianity_wiki))

In [ ]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

In [ ]:
# And now let's build useful messages for GPT-4o-mini, using a function

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
print(messages_for(History_of_Christianity_wiki))

In [ ]:
# Time to bring it together: call the OpenAI API.

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
print(summarize("https://en.wikipedia.org/wiki/History_of_Christianity"))

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://en.wikipedia.org/wiki/History_of_Christianity")